<a href="https://colab.research.google.com/github/pharringtonp19/mecon/blob/main/notebooks/practice_final/Supply_Shocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax 
import jax.numpy as jnp 

In [5]:
def output(key, q_bar, l):
  return q_bar + jax.random.normal(key)*l

In [7]:
def inverse_demand(q):
  return 12. - q**2

DeviceArray(9.794158, dtype=float32)